
# Loading and Processing of aData

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from pgmpy.estimators import BDeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel

data = pd.read_csv("HU_File_Q2_S1.csv")
data = data.drop(['Unnamed: 0', 'Calldate', 'DialStart','Calltime'], axis=1)
data

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Feature1,Feature2,Feature3,Feature4,DayOfWeek,CallCategory
0,A-,PreQ,M,SPR,Saturday,PreQ
1,A-,PreQ,M,SPR,Saturday,PostQ
2,AB+,PreQ,M,TMO,Saturday,PreQ
3,AB+,PreQ,M,TMO,Saturday,PostQ
4,AB+,PreQ,M,VER,Saturday,PreQ
...,...,...,...,...,...,...
332883,AB+,PostQ,I,TMO,Monday,PostQ
332884,A+,PostQ,I,TMO,Monday,PreQ
332885,AB+,PostQ,I,VER,Monday,PostQ
332886,AB+,PostQ,I,SPR,Monday,PreQ




# Learning Bayesian Networks


Previous notebooks showed how Bayesian networks economically encode a probability distribution over a set of variables, and how they can be used e.g. to predict variable states, or to generate new samples from the joint distribution. This section will be about obtaining a Bayesian network, given a set of sample data. Learning a Bayesian network can be split into two problems:

 **Parameter learning:** Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.
 
 **Structure learning:** Given a set of data samples, estimate a DAG that captures the dependencies between the variables.
 
This notebook aims to illustrate how parameter learning and structure learning can be done with pgmpy.
Currently, the library supports:
 - Parameter learning for *discrete* nodes:
   - Maximum Likelihood Estimation
   - Bayesian Estimation
 - Structure learning for *discrete*, *fully observed* networks:
   - Score-based structure estimation (BIC/BDeu/K2 score; exhaustive search, hill climb/tabu search)
   - Constraint-based structure estimation (PC)
   - Hybrid structure estimation (MMHC)




## Structure Learning

To learn model structure (a DAG) from a data set, there are two broad techniques:

 - score-based structure learning
 - constraint-based structure learning

The combination of both techniques allows further improvement:
 - hybrid structure learning

We briefly discuss all approaches and give examples.

### Score-based Structure Learning


This approach construes model selection as an optimization task. It has two building blocks:

- A _scoring function_ $s_D\colon M \to \mathbb R$ that maps models to a numerical score, based on how well they fit to a given data set $D$.
- A _search strategy_ to traverse the search space of possible models $M$ and select a model with optimal score.


#### Scoring functions

Commonly used scores to measure the fit between model and data are _Bayesian Dirichlet scores_ such as *BDeu* or *K2* and the _Bayesian Information Criterion_ (BIC, also called MDL). See [1], Section 18.3 for a detailed introduction on scores. As before, BDeu is dependent on an equivalent sample size.

While the scores vary slightly, we can see that the correct `model1` has a much higher score than `model2`.
Importantly, these scores _decompose_, i.e. they can be computed locally for each of the variables given their potential parents, independent of other parts of the network:

#### Search strategies
The search space of DAGs is super-exponential in the number of variables and the above scoring functions allow for local maxima. The first property makes exhaustive search intractable for all but very small networks, the second prohibits efficient local optimization algorithms to always find the optimal structure. Thus, identifiying the ideal structure is often not tractable. Despite these bad news, heuristic search strategies often yields good results.

`HillClimbSearch` implements a greedy local search that starts from the DAG `start` (default: disconnected DAG) and proceeds by iteratively performing single-edge manipulations that maximally increase the score. The search terminates once a local maximum is found.

By Single-Edge manipulations, we refer to legal operations that can be performed on the network. Operations such as addition, deletion or modification of an edge between two nodes. 


We used BicScore as our Scoring functions. 
The BIC/MDL score ("Bayesian Information Criterion", also "Minimal Descriptive Length") is a
log-likelihood score with an additional penalty for network complexity, to avoid overfitting.


In [2]:
from pgmpy.estimators import HillClimbSearch

hc = HillClimbSearch(data, scoring_method=K2Score(data))
best_model = hc.estimate()
# We used Hill Climbing to estimate the optimal Directed Acyclic Graph that best represents out model. 
print(best_model.edges()) 
## Output connected nodes/conditional dependecies, which we see that only one exists between DayOfWeek and Feature3. 
## As far as our output label is concerned: Call Category, it seems like its independent of any feature currently present in the dataset. 
## We also not that there are no dependencies between the features themselves. 

[('Feature3', 'DayOfWeek')]


In [3]:
print("Below we printed out the Leaves of our optimal DAG.")
print("In directed graphs, the common terminology is source for a node that has no incoming edges and sink for a node that has no outgoing edges. \n")

print("Leaves in optiomal DAG: ", best_model.get_leaves(), "\n")
print("We note that other than Feature 3, every other feature including our output label-CallCategory- is present as a leaf. This means that there are niether edges orignating from these features nor are there any edges approaching them, signifying: \n ")
print("1). There are no conditional dependencies between the features, Feature 3 being the exception. There is an outgoing edge from Feature 3 to DayOfWeek, which indicaties conditional dependency ")
print("2). Our output label - CallCategory - is not dependent upon any feature or a combination of features\n\n")
# print("Roots in optimal DAG: ",best_model.get_roots(), '\n')

print("To ensure point 2, we, specifically:\n")
print("A). Asked our program to check whether there are any local independencies of our CallCategory")
print(best_model.local_independencies("CallCategory"), '\n')
print("Ans). Call Category is independent of every feature.\n")

print("B). Asked our program to find all the observed variables which are d-connected to variables in our DAG when observed variables are observed.")
print(best_model.active_trail_nodes("CallCategory"), '\n')
print("Ans). There were none")

print("C). Asked our program to check whether our output label has any ancestors, that is, is it conditionally dependent upon anything?")
print(best_model._get_ancestors_of("CallCategory"), '\n')
print("Ans). No. CallCategory is dependent just on itself.")




Below we printed out the Leaves of our optimal DAG.
In directed graphs, the common terminology is source for a node that has no incoming edges and sink for a node that has no outgoing edges. 

Leaves in optiomal DAG:  ['Feature1', 'Feature2', 'Feature4', 'DayOfWeek', 'CallCategory'] 

We note that other than Feature 3, every other feature including our output label-CallCategory- is present as a leaf. This means that there are niether edges orignating from these features nor are there any edges approaching them, signifying: 
 
1). There are no conditional dependencies between the features, Feature 3 being the exception. There is an outgoing edge from Feature 3 to DayOfWeek, which indicaties conditional dependency 
2). Our output label - CallCategory - is not dependent upon any feature or a combination of features


To ensure point 2, we, specifically:

A). Asked our program to check whether there are any local independencies of our CallCategory
(CallCategory _|_ Feature1, DayOfWeek, Fea

#### (Conditional) Independence Tests

Independencies in the data can be identified using chi2 conditional independence tests. To this end, constraint-based estimators in pgmpy have a `test_conditional_independence(X, Y, Zs)`-method, that performs a hypothesis test on the data sample. It allows to check if `X` is independent from `Y` given a set of variables `Zs`.

`test_conditional_independence()` returns a tripel `(chi2, p_value, sufficient_data)`, consisting in the computed chi2 test statistic, the `p_value` of the test, and a heuristig flag that indicates if the sample size was sufficient. The `p_value` is the probability of observing the computed chi2 statistic (or an even higher chi2 value), given the null hypothesis that X and Y are independent given Zs.

This can be used to make independence judgements, at a given level of significance.

We moved onto verify our DAG obtained by carrying out some Independent test (ch2 test) to see if our results match up with independencies in our DAG.

In [4]:
from pgmpy.estimators import ConstraintBasedEstimator

 
est = ConstraintBasedEstimator(data) #Fed the Data.
# Returns result of hypothesis test for the null hypothesis that X _|_ Y | Zs, using a chi2 statistic and threshold `significance_level

print("We carried out the Chi2 test on our output label with respect to every feature. If the function returns:\n 1). True - The two nodes are independent. \n 2). False - There is a dependency between the two nodes.")
print("\n")
print(est.test_conditional_independence('CallCategory','Feature1'))          # independent
print(est.test_conditional_independence('CallCategory','Feature2'))          # independent
print(est.test_conditional_independence('CallCategory','Feature3'))          # independent
print(est.test_conditional_independence('CallCategory','Feature4'))       # independent - True is independent
print(est.test_conditional_independence('CallCategory','DayOfWeek'))      # independent
print(est.test_conditional_independence('Feature3','DayOfWeek'))          # dependent - False is dependent

print("\nOur results do match up with our optimal DAG that we learnt through Hill Climbing. Chi2 tests reveal that the output label has no dependencies whatsoever. On the contrary, a dependency exists between Feature 3 and DayOfWeek.")


We carried out the Chi2 test on our output label with respect to every feature. If the function returns:
 1). True - The two nodes are independent. 
 2). False - There is a dependency between the two nodes.


True
True
True
True
True
False

Our results do match up with our optimal DAG that we learnt through Hill Climbing. Chi2 tests reveal that the output label has no dependencies whatsoever. On the contrary, a dependency exists between Feature 3 and DayOfWeek.


## Parameter Learning



In [5]:
import pandas as pd
print(data)

       Feature1 Feature2 Feature3 Feature4 DayOfWeek CallCategory
0            A-     PreQ        M      SPR  Saturday         PreQ
1            A-     PreQ        M      SPR  Saturday        PostQ
2           AB+     PreQ        M      TMO  Saturday         PreQ
3           AB+     PreQ        M      TMO  Saturday        PostQ
4           AB+     PreQ        M      VER  Saturday         PreQ
...         ...      ...      ...      ...       ...          ...
332883      AB+    PostQ        I      TMO    Monday        PostQ
332884       A+    PostQ        I      TMO    Monday         PreQ
332885      AB+    PostQ        I      VER    Monday        PostQ
332886      AB+    PostQ        I      SPR    Monday         PreQ
332887       A+    PostQ        I      SPR    Monday        PostQ

[332888 rows x 6 columns]


We know that the variables relate as follows:

In [6]:
from pgmpy.models import BayesianModel
# model = BayesianModel([('Feature1'), ('Feature2'),('Feature3',"DayOfWeek"),('Feature4'),('CallCategory')])  # fruit -> tasty <- size# model = best_model
print("We constructed our optimal DAG manually that we learnt above. This has been done in order to learn the parameters now.")
##We constructed our optimal DAG manually that we learnt above. This has been done in order to learn the parameters now.

model = BayesianModel() 
model.add_node("Feature1")
model.add_node("Feature2")
model.add_node("Feature3")
model.add_node("Feature4")
model.add_node("DayOfWeek")
model.add_node("CallCategory")
model.add_edge("Feature3","DayOfWeek")



We constructed our optimal DAG manually that we learnt above. This has been done in order to learn the parameters now.


Parameter learning is the task to estimate the values of the conditional probability distributions (CPDs), for the variables `fruit`, `size`, and `tasty`. 

#### State counts
To make sense of the given data, we can start by counting how often each state of the variable occurs. If the variable is dependent on parents, the counts are done conditionally on the parents states, i.e. for seperately for each parent configuration:

In [7]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, data)
print("\n", pe.state_counts('CallCategory'))  # unconditional



        CallCategory
PostQ        166444
PreQ         166444


We can see, for example, that as many apples as bananas were observed and that `5` large bananas were tasty, while only `1` was not.

#### Maximum Likelihood Estimation

A natural estimate for the CPDs is to simply use the *relative frequencies*, with which the variable states have occured. We observed `7 apples` among a total of `14 fruits`, so we might guess that about `50%` of `fruits` are `apples`.

This approach is *Maximum Likelihood Estimation (MLE)*. According to MLE, we should fill the CPDs in such a way, that $P(\text{data}|\text{model})$ is maximal. This is achieved when using the *relative frequencies*. See [1], section 17.1 for an introduction to ML parameter estimation. pgmpy supports MLE as follows:

In [8]:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)

print("\nLearnt the parameters from the data. At any given point in time, there is a 50% chance that the CallCategory could either be PostQ or PreQ. This chance is also bot affected by any other features since CallCategory is completely independent.")
print(mle.estimate_cpd('CallCategory'))  
print(mle.estimate_cpd('Feature1'))  
print(mle.estimate_cpd('Feature2'))  
print(mle.estimate_cpd('Feature3'))  
print(mle.estimate_cpd('Feature4')) 




Learnt the parameters from the data. At any given point in time, there is a 50% chance that the CallCategory could either be PostQ or PreQ. This chance is also bot affected by any other features since CallCategory is completely independent.
+---------------------+-----+
| CallCategory(PostQ) | 0.5 |
+---------------------+-----+
| CallCategory(PreQ)  | 0.5 |
+---------------------+-----+
+---------------+----------+
| Feature1(A+)  | 0.142063 |
+---------------+----------+
| Feature1(A-)  | 0.285841 |
+---------------+----------+
| Feature1(AB+) | 0.428357 |
+---------------+----------+
| Feature1(AB-) | 0.143739 |
+---------------+----------+
+-----------------+----------+
| Feature2(PostQ) | 0.507065 |
+-----------------+----------+
| Feature2(PreQ)  | 0.492935 |
+-----------------+----------+
+-------------+----------+
| Feature3(F) | 0.291573 |
+-------------+----------+
| Feature3(I) | 0.303369 |
+-------------+----------+
| Feature3(M) | 0.405058 |
+-------------+----------+
+--

`mle.estimate_cpd(variable)` computes the state counts and divides each cell by the (conditional) sample size. The `mle.get_parameters()`-method returns a list of CPDs for all variable of the model.

The built-in `fit()`-method of `BayesianModel` provides more convenient access to parameter estimators:


In [9]:
# Calibrate all CPDs of `model` using MLE:
model.fit(data, estimator=MaximumLikelihoodEstimator)


While very straightforward, the ML estimator has the problem of *overfitting* to the data. In above CPD, the probability of a large banana being tasty is estimated at `0.833`, because `5` out of `6` observed large bananas were tasty. Fine. But note that the probability of a small banana being tasty is estimated at `0.0`, because we  observed only one small banana and it happened to be not tasty. But that should hardly make us certain that small bananas aren't tasty!
We simply do not have enough observations to rely on the observed frequencies. If the observed data is not representative for the underlying distribution, ML estimations will be extremly far off. 

When estimating parameters for Bayesian networks, lack of data is a frequent problem. Even if the total sample size is very large, the fact that state counts are done conditionally for each parents configuration causes immense fragmentation. If a variable has 3 parents that can each take 10 states, then state counts will be done seperately for `10^3 = 1000` parents configurations. This makes MLE very fragile and unstable for learning Bayesian Network parameters. A way to mitigate MLE's overfitting is *Bayesian Parameter Estimation*.

#### Bayesian Parameter Estimation

The Bayesian Parameter Estimator starts with already existing prior CPDs, that express our beliefs about the variables *before* the data was observed. Those "priors" are then updated, using the state counts from the observed data. See [1], Section 17.3 for a general introduction to Bayesian estimators.

One can think of the priors as consisting in *pseudo state counts*, that are added to the actual counts before normalization.
Unless one wants to encode specific beliefs about the distributions of the variables, one commonly chooses uniform priors, i.e. ones that deem all states equiprobable.

A very simple prior is the so-called *K2* prior, which simply adds `1` to the count of every single state.
A somewhat more sensible choice of prior is *BDeu* (Bayesian Dirichlet equivalent uniform prior). For BDeu we need to specify an *equivalent sample size* `N` and then the pseudo-counts are the equivalent of having observed `N` uniform samples of each variable (and each parent configuration). In pgmpy:




In [10]:
from pgmpy.estimators import BayesianEstimator
est = BayesianEstimator(model, data)

print(est.estimate_cpd('CallCategory', prior_type='BDeu', equivalent_sample_size=10))

+---------------------+-----+
| CallCategory(PostQ) | 0.5 |
+---------------------+-----+
| CallCategory(PreQ)  | 0.5 |
+---------------------+-----+


The estimated values in the CPDs are now more conservative. In particular, the estimate for a small banana being not tasty is now around `0.64` rather than `1.0`. Setting `equivalent_sample_size` to `10` means that for each parent configuration, we add the equivalent of 10 uniform samples (here: `+5` small bananas that are tasty and `+5` that aren't).

`BayesianEstimator`, too, can be used via the `fit()`-method. Full example:

In [11]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator

# generate data
model.fit(data, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)


+---------------------+-----+
| CallCategory(PostQ) | 0.5 |
+---------------------+-----+
| CallCategory(PreQ)  | 0.5 |
+---------------------+-----+
+----------------------+---------------------+---------------------+---------------------+
| Feature3             | Feature3(F)         | Feature3(I)         | Feature3(M)         |
+----------------------+---------------------+---------------------+---------------------+
| DayOfWeek(Friday)    | 0.1463615062630132  | 0.1439279737719512  | 0.13909185232376556 |
+----------------------+---------------------+---------------------+---------------------+
| DayOfWeek(Monday)    | 0.2136582355238344  | 0.21807417354816597 | 0.237000000706301   |
+----------------------+---------------------+---------------------+---------------------+
| DayOfWeek(Saturday)  | 0.08506090321618436 | 0.08467438677130099 | 0.07245023579858274 |
+----------------------+---------------------+---------------------+---------------------+
| DayOfWeek(Sunday)    | 0.0693

### References
[1] http://pgmpy.org/index.html
